In [1]:
import base64
import httpx
from langchain_core.messages import HumanMessage

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

image_url = "https://www.qgjeitinhocaseiro.com/wp-content/uploads/2019/12/comida-f%C3%A1cil.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
message = HumanMessage(
    content=[
        {"type": "text", "text": "Liste em texto absolutamente todas as comidas na imagem."},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]
)

In [2]:
model_name = "gpt-4o-2024-08-06"

llm = ChatOpenAI(model=model_name, temperature=0.01)

response = llm.invoke([message])

In [3]:
from pprint import pprint

pprint(response.content)

('Na imagem, há as seguintes comidas:\n'
 '\n'
 '- Arroz\n'
 '- Alface\n'
 '- Tomate fatiado\n'
 '- Purê de batata\n'
 '- Filé de frango com molho verde\n'
 '- Tomates cereja (ao lado do prato)')


In [11]:
from pydantic import BaseModel, Field
from typing import List
from langchain_groq import ChatGroq

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model_name_groq = "llama-3.1-70b-versatile"
model_name_openai = "gpt-4o-2024-08-06"

llm_openai = ChatOpenAI(
    model=model_name_openai, # 100% json output
    temperature=0,
)

llm_groq = ChatGroq(
    model=model_name_groq, 
    temperature=0,
)

system_prompt = """

Você é um assistente de nutrição que auxilia na listagem de alimentos dado um texto que descreve uma imagem. Registre em uma lista absolutamente todas as comidas presentes no texto.

"""

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt), 
            ("human", "Descrição da imagem: \n\n {description}")
        ]
)

class FoodList(BaseModel):
    """Gere uma lista de alimentos presentes no texto."""
    
    resultado: List[str] = Field(description="Lista de alimentos presentes na descrição da cena.", 
                                 examples=[["Banana", "Maçã", "Pão", "Queijo"],
                                           ["Arroz", "Feijão", "Carne", "Salada"]])
    

llm_openai_with_tools_extraction = llm_openai.bind_tools([FoodList]) #, strict=True)
llm_groq_with_tools_extraction = llm_groq.with_structured_output(FoodList)

chain_openai_structured_extraction = prompt | llm_openai_with_tools_extraction
chain_groq_structured_extraction = prompt | llm_groq_with_tools_extraction

In [5]:
response_struct = chain_openai_structured_extraction.invoke({"description": response.content})

In [9]:
response_struct.tool_calls[0]['args']['resultado']

['Arroz',
 'Alface',
 'Tomate fatiado',
 'Purê de batata',
 'Filé de frango com molho verde',
 'Tomates cereja']

In [12]:
response_struct_groq = chain_groq_structured_extraction.invoke({"description": response.content})

In [15]:
response_struct_groq.resultado

['Arroz',
 'Alface',
 'Tomate',
 'Purê de batata',
 'Filé de frango',
 'Tomates cereja']

In [ ]:
import base64
import httpx
from langchain_core.messages import HumanMessage

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from pydantic import BaseModel, Field
from typing import List
from langchain_groq import ChatGroq

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


model_name_groq = "llama-3.1-70b-versatile"
model_name_openai = "gpt-4o-2024-08-06"

llm_openai = ChatOpenAI(
    model=model_name_openai, # 100% json output
    temperature=0,
)

llm_groq = ChatGroq(
    model=model_name_groq, 
    temperature=0,
)

system_prompt = """

Você é um assistente de nutrição que auxilia na listagem de alimentos dado um texto que descreve uma imagem. Registre em uma lista absolutamente todas as comidas presentes no texto.

"""

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt), 
            ("human", "Descrição da imagem: \n\n {description}")
        ]
)

class FoodList(BaseModel):
    """Gere uma lista de alimentos presentes no texto."""
    
    resultado: List[str] = Field(description="Lista de alimentos presentes na descrição da cena.", 
                                 examples=[["Banana", "Maçã", "Pão", "Queijo"],
                                           ["Arroz", "Feijão", "Carne", "Salada"]])
    

llm_openai_with_tools_extraction = llm_openai.bind_tools([FoodList]) #, strict=True)
llm_groq_with_tools_extraction = llm_groq.with_structured_output(FoodList)

chain_openai_structured_extraction = prompt | llm_openai_with_tools_extraction
chain_groq_structured_extraction = prompt | llm_groq_with_tools_extraction


def get_structured_foods(path: str, provider="openai"):
    image_url = path # "https://www.qgjeitinhocaseiro.com/wp-content/uploads/2019/12/comida-f%C3%A1cil.jpg"
    image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
    message = HumanMessage(
        content=[
            {"type": "text", "text": "Liste em texto absolutamente todas as comidas na imagem."},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
            },
        ]
    )
    
    response = llm.invoke([message])
    response_content = response.content
    
    if provider == "openai":
        response_struct = chain_openai_structured_extraction.invoke({"description": response_content})
        try:
            output_struct = response_struct.tool_calls[0]['args']['resultado']
        except:
            output_struct = []
    elif provider == "groq":
        response_struct = chain_groq_structured_extraction.invoke({"description": response_content})
        try:
            output_struct = response_struct.resultado
        except:
            output_struct = []

    return output_struct

In [17]:
image_url_1 = "https://institucional.ifood.com.br/wp-content/uploads/2024/07/comidas-tipicas-arroz-com-pequi.jpg" # arroz com pequi
image_url_2 = "https://institucional.ifood.com.br/wp-content/uploads/2024/07/comidas-tipicas-moqueca.jpg" # moqueca arroz cebola ....
image_url_3 = "https://institucional.ifood.com.br/wp-content/uploads/2024/07/comidas-tipicas-brigadeiro.jpg" # brigadeiro
image_url_4 = "https://institucional.ifood.com.br/wp-content/uploads/2024/07/pratos-tipicos-frango-com-quiabo.jpg" # frango com quiabo

images_urls = [image_url_1, image_url_2, image_url_3, image_url_4]


In [19]:
from tqdm import tqdm

all_foods = []
for url in tqdm(images_urls):
    response = get_structured_foods(url, provider="openai")
    all_foods.append(response)
    print(response)

 25%|██▌       | 1/4 [00:04<00:13,  4.55s/it]

['Arroz', 'Pequi', 'Salsa']


 50%|█████     | 2/4 [00:09<00:09,  4.53s/it]

['Moqueca de camarão', 'Arroz branco', 'Molho ou pirão', 'Pimentões vermelho', 'Pimentões amarelo', 'Cebolas', 'Alho', 'Coentro', 'Pimenta rosa']


 75%|███████▌  | 3/4 [00:12<00:03,  3.87s/it]

['Brigadeiro', 'Granulado de chocolate']


100%|██████████| 4/4 [00:16<00:00,  4.03s/it]

['Frango', 'Quiabo', 'Polenta', 'Molho de tomate', 'Queijo ralado']
